In [2]:
# import torch
# import yaml
# import os

# ckpt_dir = '/home/s1973609/repos/s3prl/s3prl/hubModels/'
# ckpt_file = os.path.join(ckpt_dir, 'distilhubert_official.ckpt')
# all_states = torch.load(ckpt_file, map_location='cpu')

# yaml_out_path = os.path.join(ckpt_dir, 'distiller_sample_config.yaml')
# with open(yaml_out_path, 'w') as f:
#     yaml.dump(all_states['Config']['distiller'], f)

/home/s1973609/miniconda3/envs/EXPR/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---------------------

In [1]:
import os
from sys import getsizeof
import yaml
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')

import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

/home/s1973609/miniconda3/envs/EXPR/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pretrain.attnDistiller.dataset import OnlineWaveDataset

from upstream.attnDistiller.model import DistillerModel, DistillerConfig
from upstream.attnDistiller.expert import UpstreamExpert as DistilExpert

from upstream.attnHuBERT.convert import load_converted_model
from upstream.attnHuBERT.expert import UpstreamExpert as HuBERTExpert

In [3]:
# device = 'cuda'

model_dir = '/home/s1973609/repos/s3prl/s3prl/hubModels/'
dataPath = '/home/s1973609/librispeech/LibriSpeechLocal'
upstreamConfigFile = 'pretrain/attnDistiller/config_model.yaml'

In [4]:
print("-------------- Loading Data ------------------------------")
dataset = OnlineWaveDataset(
    task_config={'sequence_length':250000},
    bucket_size=1,
    file_path=dataPath,
    libri_root=dataPath,
    sets=['train-clean-100'],
    target_level=None,
)

dataloader = DataLoader(
    dataset,
    batch_size=1,  # for bucketing
    shuffle=False,
    num_workers=4,
    drop_last=False,
    pin_memory=True,
    collate_fn=dataset.collate_fn,
)

sample_batch = None
for i, batch in enumerate(dataloader):
    if i == 5:
        sample_batch = batch
        break
print("Sample batch Loaded!")


wave_input, wave_orig, wave_len, pad_mask = sample_batch
# wave_input = wave_input.to(device)
# wave_len   = wave_len.to(device)
# pad_mask   = pad_mask.type(wave_input.dtype).to(device)

# wave_input - tensor with shape (batch_size, seq_len)
# wave_orig  - list of tensors, with length = batch_size and each tensor being (seq_len)
# wave_len   - tensor with shape (batch_size), each element being length
# pad_mask   - tensor with shape (batch_size, seq_len), each element being 1 or 0 (masks)
print("Shapes of elements in the sample batch:")
print(wave_input.size())
print(wave_orig[0].size())
print(wave_len.size())
print(pad_mask.size())

-------------- Loading Data ------------------------------
[Dataset] - Sampling random segments for training, sample length: 250000
[Dataset] - Training data from these sets: ['train-clean-100']
[Dataset] - Number of individual training instances: 111
Sample batch Loaded!
Shapes of elements in the sample batch:
torch.Size([1, 250000])
torch.Size([250000])
torch.Size([1])
torch.Size([1, 250000])


In [5]:
print("-------------- Loading Distiller -------------------------")
distillerConfig = yaml.load(
                open(upstreamConfigFile, "r"), Loader=yaml.FullLoader
            )
modelConfig = DistillerConfig(distillerConfig['distiller'])
distiller = DistillerModel(modelConfig)

-------------- Loading Distiller -------------------------
[DistillerModel] - Expands the output dimension by 2 times
[DistillerModel] - Pred layers: [4, 12]
[TransformerEncoder] - Attention type = original
[DistillerModel] - Out layer type: expand-last
[DistillerModel] - Inter dim = 768


In [6]:
print("-------------- Feeding Data to Distiller ---------------------------")
attn_selected_student = [1, 2]
with torch.no_grad():
    feat, feat_final, pred, pad_mask, attnMapsDistiller = distiller(wave_input, 
                                                pad_mask,
                                                attn_selected=attn_selected_student)

-------------- Feeding Data to Distiller ---------------------------


NameError: name 'attn_selected' is not defined

torch.Size([1, 781, 768])